# NetSquid in 10 minutes
https://docs.netsquid.org/latest-release/quick_start.html

In [1]:
import netsquid as ns

## Qubits

In [2]:
qubits = ns.qubits.create_qubits(1)

In [3]:
qubits

[Qubit('QS#0-0')]

In [4]:
qubit = qubits[0]
ns.qubits.reduced_dm(qubit)

array([[1.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j]])

In [5]:
ns.qubits.operate(qubit, ns.X)
ns.qubits.reduced_dm(qubit)  # reduced density matrix

array([[0.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j]])

In [6]:
# measure in standard basis (Z)
measurement_result, prob = ns.qubits.measure(qubit)

In [7]:
if measurement_result == 0:
    state = "|0>"
else:
    state = "|1>"
print(f"measured {state} with probability {prob:.1f}")

measured |1> with probability 1.0


In [8]:
# measure in Hadamard (X) basis
measurement_result, prob = ns.qubits.measure(qubit, observable=ns.X)

In [9]:
if measurement_result == 0:
    state = '|+>'
else:
    state = '|->'
print(f'measured {state} with probability {prob:.1f}')

measured |-> with probability 0.5


In [10]:
ns.qubits.reduced_dm(qubit)

array([[ 0.5+0.j, -0.5+0.j],
       [-0.5+0.j,  0.5+0.j]])

# Simulation engine

## Components

In [11]:
from netsquid.nodes import Node
node_ping = Node(name = 'Ping')
node_pong = Node(name = 'Pong')

In [12]:
from netsquid.components.models import DelayModel

class PingPongDelayModel(DelayModel):
    def __init__(self, speed_of_light_fraction=0.5, standard_deviation=0.5):
        super().__init__()
        self.properties['speed'] = speed_of_light_fraction * 3e5
        self.properties['std']   = standard_deviation
        self.required_properties = ['length']

    def generate_delay(self, **kwargs):
        avg_speed = self.properties['speed']
        std = self.properties['std']
        speed = self.properties['rng'].normal(avg_speed, avg_speed * std)
        delay = 1e9 * kwargs['length'] / speed
        return delay

In [13]:
from netsquid.components import QuantumChannel

distance = 2.74 / 1000
delay_model = PingPongDelayModel()
channel_1 = QuantumChannel(name='qchannel[ping to pong]', length=distance, models={'delay_model': delay_model})
channel_2 = QuantumChannel(name='qchannel[pong to ping]', length=distance, models={'delay_model': delay_model})

In [14]:
from netsquid.nodes import DirectConnection

connection = DirectConnection(name='conn[ping|pong]', channel_AtoB=channel_1, channel_BtoA=channel_2)
node_ping.connect_to(remote_node=node_pong, connection=connection, local_port_name='qubitIO', remote_port_name='qubitIO')

('qubitIO', 'qubitIO')

## Protocols

In [15]:
from netsquid.protocols import NodeProtocol

In [16]:
class PingPongProtocol(NodeProtocol):
    def __init__(self, node, observable, qubit=None):
        super().__init__(node)
        self.observable = observable
        self.qubit = qubit
        self.basis = ['|0>', '|1>'] if observable == ns.Z else ['|+>', '|->']

    def run(self):
        if self.qubit is not None:
            # send (TX) qubit to the other node via port's output
            self.node.ports['qubitIO'].tx_output(self.qubit)
        while True:
            # wait (yield) until input has arrived on our port
            yield self.await_port_input(self.node.ports['qubitIO'])
            # Receive (RX) qubit on the port's input
            message = self.node.ports['qubitIO'].rx_input()
            qubit = message.items[0]
            meas, prob = ns.qubits.measure(qubit, observable=self.observable)
            print(f'{ns.sim_time():5.1f}: {self.node.name} measured {self.basis[meas]} with probability {prob:.2f}')
            # send (TX) qubit to the other node via connection
            self.node.ports['qubitIO'].tx_output(qubit)

In [17]:
qubits = ns.qubits.create_qubits(1)
ping_protocol = PingPongProtocol(node_ping, observable=ns.Z, qubit=qubits[0])
pong_protocol = PingPongProtocol(node_pong, observable=ns.X)

In [18]:
ping_protocol.start()
pong_protocol.start()

PingPongProtocol(Node(name='Pong'), 'PingPongProtocol')

In [19]:
run_stats = ns.sim_run(duration=300)

 23.2: Pong measured |+> with probability 0.50
 36.5: Ping measured |0> with probability 0.50
 52.6: Pong measured |-> with probability 0.50
 78.1: Ping measured |0> with probability 0.50
131.8: Pong measured |+> with probability 0.50
194.3: Ping measured |0> with probability 0.50
210.6: Pong measured |-> with probability 0.50
231.7: Ping measured |1> with probability 0.50
247.6: Pong measured |-> with probability 0.50
266.1: Ping measured |1> with probability 0.50
280.8: Pong measured |-> with probability 0.50
295.3: Ping measured |1> with probability 0.50
